In [1]:
import sqlite3
import pandas as pd
import numpy as np
# creating file path
dbfile = 'TRACE.db'
# Create a SQL connection to our SQLite database
conn = sqlite3.connect(dbfile)

# creating cursor
cursor = conn.cursor()

In [2]:
# reading all table names
#table_list = [a for a in cursor.execute("SELECT name FROM sqlite_master WHERE type = 'table'")]
#query = "SELECT * FROM daily_btds"
#df=(cursor.execute(query)).fetchall()
#column_headers = [description[0] for description in cursor.description]

In [3]:
bond_firm_factors = pd.read_csv('bond_firm_factors.csv')
fin_ratio = pd.read_csv('wrds_fin_ratio.csv')

Fin_ratio columns description:

1. gvkey: Global company key
2. permno: Permanent number assigned by CRSP
3. adate: Announcement date
4. qdate: Quarter end date
5. public_date: Public date
6. CAPEI: Cyclically adjusted price-to-earnings ratio
7. bm: Book-to-market ratio
8. evm: Enterprise value to sales ratio
9. pe_op_basic: Operating income basic earnings per share
10. pe_op_dil: Operating income diluted earnings per share
11. pe_exi: Earnings per share excluding extraordinary items
12. pe_inc: Earnings per share including extraordinary items
13. ps: Price-to-sales ratio
14. pcf: Price-to-cash flow ratio
15. dpr: Dividend payout ratio
16. npm: Net profit margin
17. opmbd: Operating profit margin before depreciation
18. opmad: Operating profit margin after depreciation
19. gpm: Gross profit margin
20. ptpm: Pretax profit margin
21. cfm: Cash flow margin
22. roa: Return on assets
23. roe: Return on equity
24. roce: Return on capital employed
25. efftax: Effective tax rate
26. aftret_eq: After-tax return on equity
27. aftret_invcapx: After-tax return on invested capital
28. aftret_equity: After-tax return on equity
29. pretret_noa: Pretax return on net operating assets
30. pretret_earnat: Pretax return on total assets
31. GProf: Gross profit to sales ratio
32. equity_invcap: Equity to invested capital ratio
33. debt_invcap: Debt to invested capital ratio
34. totdebt_invcap: Total debt to invested capital ratio
35. capital_ratio: Capital ratio
36. int_debt: Interest-bearing debt to total capital ratio
37. int_totdebt: Interest-bearing total debt to total capital ratio
38. cash_lt: Long-term cash ratio
39. invt_act: Inventory turnover ratio
40. rect_act: Receivables turnover ratio
41. debt_at: Debt to total assets ratio
42. debt_ebitda: Debt to EBITDA ratio
43. short_debt: Short-term debt to total capital ratio
44. curr_debt: Current debt to total capital ratio
45. lt_debt: Long-term debt to total capital ratio
46. profit_lct: Profit to liquidity ratio
47. ocf_lct: Operating cash flow to liquidity ratio
48. cash_debt: Cash to debt ratio
49. fcf_ocf: Free cash flow to operating cash flow ratio
50. lt_ppent: Long-term debt to property, plant, and equipment ratio
51. dltt_be: Long-term debt to total equity ratio
52. debt_assets: Debt to assets ratio
53. debt_capital: Debt to capital ratio
54. de_ratio: Debt equity ratio
55. intcov: Interest coverage ratio
56. intcov_ratio: Interest coverage ratio
57. cash_ratio: Cash ratio
58. quick_ratio: Quick ratio
59. curr_ratio: Current ratio
60. cash_conversion: Cash conversion cycle
61. inv_turn: Inventory turnover ratio
62. at_turn: Asset turnover ratio
63. rect_turn: Receivables turnover ratio
64. pay_turn: Payables turnover ratio
65. sale_invcap: Sales to invested capital ratio
66. sale_equity: Sales to equity ratio
67. sale_nwc: Sales to net working capital ratio
68. rd_sale: Research and development to sales ratio
69. adv_sale: Advertising to sales ratio
70. staff_sale: Staff to sales ratio
71. accrual: Accruals
72. ptb: Price-to-book ratio
73. PEG_trailing: Trailing price/earnings to growth ratio
74. divyield: Dividend yield
75. TICKER: Stock ticker symbol
76. cusip: CUSIP number



In [4]:
fin_ratio.columns

Index(['gvkey', 'permno', 'adate', 'qdate', 'public_date', 'CAPEI', 'bm',
       'evm', 'pe_op_basic', 'pe_op_dil', 'pe_exi', 'pe_inc', 'ps', 'pcf',
       'dpr', 'npm', 'opmbd', 'opmad', 'gpm', 'ptpm', 'cfm', 'roa', 'roe',
       'roce', 'efftax', 'aftret_eq', 'aftret_invcapx', 'aftret_equity',
       'pretret_noa', 'pretret_earnat', 'GProf', 'equity_invcap',
       'debt_invcap', 'totdebt_invcap', 'capital_ratio', 'int_debt',
       'int_totdebt', 'cash_lt', 'invt_act', 'rect_act', 'debt_at',
       'debt_ebitda', 'short_debt', 'curr_debt', 'lt_debt', 'profit_lct',
       'ocf_lct', 'cash_debt', 'fcf_ocf', 'lt_ppent', 'dltt_be', 'debt_assets',
       'debt_capital', 'de_ratio', 'intcov', 'intcov_ratio', 'cash_ratio',
       'quick_ratio', 'curr_ratio', 'cash_conversion', 'inv_turn', 'at_turn',
       'rect_turn', 'pay_turn', 'sale_invcap', 'sale_equity', 'sale_nwc',
       'rd_sale', 'adv_sale', 'staff_sale', 'accrual', 'ptb', 'PEG_trailing',
       'divyield', 'TICKER', 'cusip'],
  

In [5]:

master = pd.read_csv('master_corp_agency.csv')
df_btds = pd.read_csv('daily_btds_144A.csv')

In [6]:


master['monthly_cpn']=master['cpn_rt']/12

# Filtering for Investment Grade Bonds
master_inv=master.loc[master['grade']=='I']
btds_cusips=pd.DataFrame(df_btds['cusip_id'].unique())
master_cusips=pd.DataFrame(master_inv['cusip_id'].unique())
merged_df=df_btds.merge(master_inv,on='cusip_id')
merged_df['close_pr']=merged_df['close_pr']+merged_df['monthly_cpn']
merged_df['mtrty_dt'].sort_values(ascending=False)
merged_df_active=merged_df
bond_prices=merged_df_active[['trans_dt','cusip_id','close_pr']]



In [7]:
# Convert trans_dt to datetime
bond_prices['trans_dt'] = pd.to_datetime(bond_prices['trans_dt'])

# Ensure data is sorted
bond_prices = bond_prices.sort_values(by=['cusip_id', 'trans_dt'])

# Remove duplicates by taking the mean or last (here we use mean for example)
bond_prices = bond_prices.groupby(['trans_dt', 'cusip_id']).mean().reset_index()

# Set the index to trans_dt
bond_prices.set_index('trans_dt', inplace=True)

# Find the min and max dates to create a full date range
min_date = bond_prices.index.min()
max_date = bond_prices.index.max()
date_range = pd.date_range(min_date, max_date, freq='D')

# Reindex the dataframe for each cusip_id to include all days in the range
# and forward fill the missing data
bond_prices = (
    bond_prices
    .groupby('cusip_id')
    .apply(lambda x: x.reindex(date_range).ffill().reset_index())
    .reset_index(drop=True)
)

# Rename 'index' back to 'trans_dt'
bond_prices.rename(columns={'index': 'trans_dt'}, inplace=True)

# Calculate log returns
bond_prices['log_returns'] = np.log(bond_prices['close_pr'] / bond_prices['close_pr'].shift(1))

# Average log returns by cusip_id
# average_log_returns = bond_prices.groupby('cusip_id')['log_returns'].mean().reset_index()



/var/folders/b_/cdyf_8w5367dll1wdwdy17p40000gn/T/ipykernel_2937/3900696667.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bond_prices['trans_dt'] = pd.to_datetime(bond_prices['trans_dt'])
/var/folders/b_/cdyf_8w5367dll1wdwdy17p40000gn/T/ipykernel_2937/3900696667.py:21: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  bond_prices


In [8]:
# Calculate yearly log returns
bond_prices['weekly'] = bond_prices['trans_dt'].dt.to_period('W')
weekly_log_returns = bond_prices.groupby(['cusip_id', 'weekly'])['log_returns'].sum().reset_index()

# Calculate monthly log returns
bond_prices['month_year'] = bond_prices['trans_dt'].dt.to_period('M')
monthly_log_returns = bond_prices.groupby(['cusip_id', 'month_year'])['log_returns'].sum().reset_index()

# Calculate quarterl log returns
bond_prices['quarterly'] = bond_prices['trans_dt'].dt.to_period('Q')
quarterly_log_returns = bond_prices.groupby(['cusip_id', 'quarterly'])['log_returns'].sum().reset_index()

# Calculate yearly log returns
bond_prices['year'] = bond_prices['trans_dt'].dt.to_period('Y')
yearly_log_returns = bond_prices.groupby(['cusip_id', 'year'])['log_returns'].sum().reset_index()

In [9]:
weekly_log_returns
monthly_log_returns
quarterly_log_returns
yearly_log_returns



,cusip_id,year,log_returns
0,00036AAB1,2015,-0.072406
1,00036AAB1,2016,0.049706
2,00036AAB1,2017,0.000000
3,00036AAB1,2018,-0.041977
4,00036AAB1,2019,0.024144
...,...,...,...
48408,Y0R9K1AC4,2022,-0.068894
48409,Y0R9K1AC4,2023,0.001177
48410,Y20339AG5,2021,0.000000
48411,Y20339AG5,2022,0.000000


In [10]:
bond_prices

,trans_dt,cusip_id,close_pr,log_returns,weekly,month_year,quarterly,year
0,2014-12-31,NaN,NaN,NaN,2014-12-29/2015-01-04,2014-12,2014Q4,2014
1,2015-01-01,NaN,NaN,NaN,2014-12-29/2015-01-04,2015-01,2015Q1,2015
2,2015-01-02,NaN,NaN,NaN,2014-12-29/2015-01-04,2015-01,2015Q1,2015
3,2015-01-03,NaN,NaN,NaN,2014-12-29/2015-01-04,2015-01,2015Q1,2015
4,2015-01-04,NaN,NaN,NaN,2014-12-29/2015-01-04,2015-01,2015Q1,2015
...,...,...,...,...,...,...,...,...
26216347,2023-09-24,Y20339AG5,101.256333,0.0,2023-09-18/2023-09-24,2023-09,2023Q3,2023
26216348,2023-09-25,Y20339AG5,101.256333,0.0,2023-09-25/2023-10-01,2023-09,2023Q3,2023
26216349,2023-09-26,Y20339AG5,101.256333,0.0,2023-09-25/2023-10-01,2023-09,2023Q3,2023
26216350,2023-09-27,Y20339AG5,101.256333,0.0,2023-09-25/2023-10-01,2023-09,2023Q3,2023


In [12]:

# 1. Profitability Growth Rate
fin_ratio['profit_growth_rate'] = fin_ratio['npm'].pct_change()

# 2. Liquidity Ratio Dynamics
fin_ratio['cop_at'] = fin_ratio['opmbd'] / fin_ratio['equity_invcap'] # (Cash-based operating profits-to-book assets)
fin_ratio['noa_gr1a'] = fin_ratio['roa'].diff() #  (Change in net operating assets)
fin_ratio['saleq_gr1'] = fin_ratio['sale_invcap'].pct_change() # (Sales growth (1 quarter))
fin_ratio['curr_ratio_change'] = fin_ratio['curr_ratio'].diff()
fin_ratio['quick_ratio_change'] = fin_ratio['quick_ratio'].diff()
fin_ratio['cash_ratio_change'] = fin_ratio['cash_ratio'].diff()

# 3. Debt Management Efficiency
fin_ratio['debt_me'] = fin_ratio['totdebt_invcap'] / fin_ratio['evm'] # debt to market
fin_ratio['debt_assets_change'] = fin_ratio['debt_assets'].diff()
fin_ratio['de_ratio_change'] = fin_ratio['de_ratio'].diff()

# 4. Capital Structure Stability
fin_ratio['debt_capital_change'] = fin_ratio['debt_capital'].diff()
fin_ratio['debt_invcap_change'] = fin_ratio['debt_invcap'].diff()

# 5. Earnings Quality Metrics
fin_ratio['recurring_earnings_ratio'] = fin_ratio['opmad'] / fin_ratio['opmbd']
fin_ratio['earnings_growth_consistency'] = fin_ratio['pe_op_basic'].pct_change()

# 6. Investment Efficiency Ratios
fin_ratio['roce_change'] = fin_ratio['roce'].diff()
fin_ratio['aftret_invcapx_change'] = fin_ratio['aftret_invcapx'].diff()

# 7. Market Sentiment Indicators
fin_ratio['ptb_change'] = fin_ratio['ptb'].diff()
fin_ratio['peg_trailing_change'] = fin_ratio['PEG_trailing'].diff()

# 8. Risk-adjusted Performance Measures
fin_ratio['efftax_change'] = fin_ratio['efftax'].diff()

#9 Operating capital Indicator 
fin_ratio['cowc_gr1a'] = fin_ratio['ocf_lct'].diff() # Change in current operating working capital
fin_ratio['nncoa_gr1a'] = fin_ratio['aftret_invcapx'].diff() #Change in net noncurrent operating assets
fin_ratio['ocf_me'] = fin_ratio['ocf_lct'] / fin_ratio['evm'] #Operating cash flow-to-market




/var/folders/b_/cdyf_8w5367dll1wdwdy17p40000gn/T/ipykernel_2937/1211590327.py:2: FutureWarning: The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  fin_ratio['profit_growth_rate'] = fin_ratio['npm'].pct_change()
/var/folders/b_/cdyf_8w5367dll1wdwdy17p40000gn/T/ipykernel_2937/1211590327.py:7: FutureWarning: The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  fin_ratio['saleq_gr1'] = fin_ratio['sale_invcap'].pct_change() # (Sales growth (1 quarter))
/var/folders/b_/cdyf_8w5367dll1wdwdy17p40000gn/T/ipykernel_2937/1211590327.py:23: FutureWarning: The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a futur

In [13]:
bond_firm_factors.columns


Index(['date', 'Value', 'Market Value*', 'Amount Outstanding*', 'Duration*',
       'Bond Age', 'Book-to-price', 'Credit Spread', 'Yield to maturity',
       'VaR(5%)', 'Volatility', 'ES(10%)', 'Default-beta', 'Term-beta',
       'VaR(10%)', 'Skewness*', '3m. Momentum', '6m. Momentum', '9m. Momentum',
       '12m. Momentum', 'Equity Momentum', 'Short Term Reversal',
       'Long Term Reversal'],
      dtype='object')

Documenting the bond credit factors from the bond_firm_factor dataset:

1. **Market Value*:** The current market value of the bond, indicating its liquidity and demand in the market.

2. **Amount Outstanding*:** The total amount of the bond outstanding, providing insight into its popularity and issuance volume.

3. **Duration*:** The bond's duration, representing its sensitivity to changes in interest rates and potential price fluctuations.

4. **Bond Age:** The length of time since the bond was issued, influencing its maturity and historical performance.

5. **Book-to-Price:** The ratio of the bond's book value to its market price, indicating its valuation relative to its accounting value.

6. **Credit Spread:** The difference in yield between the bond and a comparable risk-free asset, reflecting the bond's credit risk.

7. **Yield to Maturity:** The total return anticipated on the bond if held until maturity, considering both coupon payments and capital gains or losses.

8. **VaR(5%) and VaR(10%):** Value at Risk measures at different confidence levels, providing insight into potential downside risk.

9. **Volatility:** The magnitude of the bond's price fluctuations over a certain period, indicating its risk level.

10. **ES(10%):** Expected shortfall at a 10% confidence level, offering a measure of the bond's potential losses beyond VaR.

11. **Default-beta:** The bond's sensitivity to changes in default risk, reflecting its susceptibility to credit events.

12. **Term-beta:** The bond's sensitivity to changes in interest rates, especially concerning its maturity term.

13. **Skewness*:** The asymmetry of the bond's return distribution, indicating the likelihood of extreme positive or negative returns.

14. **Momentum (3m, 6m, 9m, 12m):** The bond's recent performance trend over different time horizons, influencing investor sentiment and expectations.

15. **Equity Momentum:** The momentum of related equity securities, providing insights into market sentiment and systemic risk factors.

16. **Short Term Reversal:** The bond's tendency to reverse short-term price movements, potentially indicating overbought or oversold conditions.

17. **Long Term Reversal:** Similar to short-term reversal but over longer timeframes, reflecting broader market trends and investor sentiment shifts.

These factors collectively provide a comprehensive view of a bond's creditworthiness, risk profile, and potential for investment returns.

In [14]:
####### Bond Like Factors  ######

# Factor 1: Duration Sensitivity to Credit Spread Changes
bond_firm_factors['Duration_Spread_Sensitivity'] = bond_firm_factors['Duration*'] * bond_firm_factors['Credit Spread']

# Factor 2: Yield Volatility Ratio
bond_firm_factors['Yield_Volatility_Ratio'] = bond_firm_factors['Volatility'] / bond_firm_factors['Market Value*']

# Factor 3: Momentum-based Duration Adjustment
bond_firm_factors['Momentum_Adjusted_Duration'] = bond_firm_factors['Duration*'] * (
        1 + (bond_firm_factors['3m. Momentum'] + bond_firm_factors['6m. Momentum'] +
             bond_firm_factors['9m. Momentum'] + bond_firm_factors['12m. Momentum']) / 4
)

# Factor 4: Term Structure Risk Factor
bond_firm_factors['Term_Structure_Risk'] = (bond_firm_factors['VaR(5%)'] - bond_firm_factors['VaR(10%)']) / bond_firm_factors['Bond Age']

# Factor 5: Credit Spread Sensitivity to Default Risk
bond_firm_factors['Credit_Spread_Default_Sensitivity'] = bond_firm_factors['Credit Spread'] / bond_firm_factors['Default-beta']

# Factor 6: Momentum-based Yield Adjustment
bond_firm_factors['Momentum_Adjusted_Yield'] = bond_firm_factors['Yield to maturity'] * (
        1 + (bond_firm_factors['3m. Momentum'] + bond_firm_factors['6m. Momentum'] +
             bond_firm_factors['9m. Momentum'] + bond_firm_factors['12m. Momentum']) / 4
)

# Factor 7: Market Value Volatility Ratio
bond_firm_factors['Market_Value_Volatility_Ratio'] = bond_firm_factors['Volatility'] / bond_firm_factors['Book-to-price']

# Factor 8: Skewness-based Yield Adjustment
bond_firm_factors['Skewness_Adjusted_Yield'] = bond_firm_factors['Yield to maturity'] * (1 + bond_firm_factors['Skewness*'])

"""
Innovative Fixed Income Factors Implementation

1. Duration Sensitivity to Credit Spread Changes:
   - Measures how the duration of a bond reacts to changes in credit spreads.
   - Formula: Duration_Spread_Sensitivity = Duration * Credit Spread

2. Yield Volatility Ratio:
   - Quantifies the volatility of bond yields relative to their market value.
   - Formula: Yield_Volatility_Ratio = Volatility / Market Value

3. Momentum-based Duration Adjustment:
   - Considers the momentum of a bond's market value over different time periods and adjusts the bond's duration accordingly.
   - Formula: Momentum_Adjusted_Duration = Duration * (1 + (3m. Momentum + 6m. Momentum + 9m. Momentum + 12m. Momentum) / 4)

4. Term Structure Risk Factor:
   - Captures the risk associated with changes in the term structure of interest rates.
   - Formula: Term_Structure_Risk = (VaR(5%) - VaR(10%)) / Bond Age

5. Credit Spread Sensitivity to Default Risk:
   - Measures the sensitivity of credit spreads to changes in default risk.
   - Formula: Credit_Spread_Default_Sensitivity = Credit Spread / Default-beta

6. Momentum-based Yield Adjustment:
   - Adjusts the bond's yield to maturity based on its momentum over different time horizons.
   - Formula: Momentum_Adjusted_Yield = Yield to maturity * (1 + (3m. Momentum + 6m. Momentum + 9m. Momentum + 12m. Momentum) / 4)

7. Market Value Volatility Ratio:
   - Evaluates the volatility of a bond's market value relative to its book-to-price ratio.
   - Formula: Market_Value_Volatility_Ratio = Volatility / Book-to-price

8. Skewness-based Yield Adjustment:
   - Adjusts the bond's yield to maturity based on its skewness.
   - Formula: Skewness_Adjusted_Yield = Yield to maturity * (1 + Skewness)
"""


"\nInnovative Fixed Income Factors Implementation\n\n1. Duration Sensitivity to Credit Spread Changes:\n   - Measures how the duration of a bond reacts to changes in credit spreads.\n   - Formula: Duration_Spread_Sensitivity = Duration * Credit Spread\n\n2. Yield Volatility Ratio:\n   - Quantifies the volatility of bond yields relative to their market value.\n   - Formula: Yield_Volatility_Ratio = Volatility / Market Value\n\n3. Momentum-based Duration Adjustment:\n   - Considers the momentum of a bond's market value over different time periods and adjusts the bond's duration accordingly.\n   - Formula: Momentum_Adjusted_Duration = Duration * (1 + (3m. Momentum + 6m. Momentum + 9m. Momentum + 12m. Momentum) / 4)\n\n4. Term Structure Risk Factor:\n   - Captures the risk associated with changes in the term structure of interest rates.\n   - Formula: Term_Structure_Risk = (VaR(5%) - VaR(10%)) / Bond Age\n\n5. Credit Spread Sensitivity to Default Risk:\n   - Measures the sensitivity of cre

In [15]:
bond_firm_factors


,date,Value,Market Value*,Amount Outstanding*,Duration*,Bond Age,Book-to-price,Credit Spread,Yield to maturity,VaR(5%),...,Short Term Reversal,Long Term Reversal,Duration_Spread_Sensitivity,Yield_Volatility_Ratio,Momentum_Adjusted_Duration,Term_Structure_Risk,Credit_Spread_Default_Sensitivity,Momentum_Adjusted_Yield,Market_Value_Volatility_Ratio,Skewness_Adjusted_Yield
0,1985-02-28,-0.008193,0.010543,0.010151,0.017169,0.003792,-0.010576,0.004922,0.000062,-0.017357,...,NaN,-0.003509,8.450844e-05,-1.185854,NaN,-0.792685,0.488565,NaN,1.182253,0.000062
1,1985-03-31,-0.001512,-0.003335,-0.003432,-0.002773,-0.001762,0.002099,0.003432,0.003400,0.004021,...,0.005045,0.003433,-9.516397e-06,-1.189973,NaN,-1.019325,-8.627481,NaN,1.890915,0.003410
2,1985-04-30,0.000291,0.001172,0.000454,-0.000171,-0.000925,0.003448,0.000760,0.005383,-0.004547,...,0.006169,-0.005118,-1.301319e-07,3.801382,NaN,5.582723,-0.496205,NaN,1.291671,0.005337
3,1985-05-31,0.006711,-0.007488,-0.009067,-0.024065,0.003893,0.020272,0.005627,0.009315,0.030455,...,-0.005399,0.004307,-1.354093e-04,-2.982229,NaN,2.413415,-0.638222,NaN,1.101620,0.009335
4,1985-06-30,-0.004259,0.003979,0.003116,0.006696,0.000974,-0.001852,0.003520,0.001228,0.000862,...,0.005181,0.001013,2.356770e-05,-0.443999,NaN,5.044117,2.919082,NaN,0.953866,0.001229
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
438,2021-08-31,-0.000400,0.000711,0.000495,0.002417,0.001611,0.002055,0.001192,0.000066,-0.000109,...,0.003723,0.003251,2.880440e-06,-1.348542,0.002419,0.908243,3.814143,0.000066,-0.466446,0.000066
439,2021-09-30,-0.000134,0.004892,0.004596,0.008656,0.007650,0.006333,0.003711,-0.001178,-0.002593,...,-0.004984,0.007086,3.211925e-05,-0.102627,0.008655,0.361768,0.880623,-0.001177,-0.079266,-0.001184
440,2021-10-31,-0.005729,-0.001648,-0.001314,-0.007370,-0.004876,-0.004047,0.003167,0.007153,0.007951,...,0.004506,-0.002763,-2.334006e-05,-4.659711,-0.007396,0.069298,1.453486,0.007179,-1.897572,0.007140
441,2021-11-30,0.002358,0.000713,0.000950,-0.004310,-0.003855,-0.001402,-0.002489,-0.001086,-0.001416,...,-0.002344,-0.002096,1.072877e-05,-0.776933,-0.004299,0.659111,0.827958,-0.001083,0.395113,-0.001086
